# 前置作業

In [ ]:
# Mount to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define Project Folder
FOLDERNAME = 'Colab\ Notebooks/Group_A_Project'

%cd drive/MyDrive/$FOLDERNAME

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.cuda.amp import autocast, GradScaler
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, pipeline
# from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils.class_weight import compute_class_weight
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr
from collections import defaultdict
from imblearn.over_sampling import BorderlineSMOTE
from wordcloud import WordCloud

In [ ]:
# Define device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Seed for same output
torch.manual_seed(42)  # the random numbers generated (e.g., for initializing weights) are the same each time the code is run
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# 資料前處理

In [ ]:
# Reading in our file
raw_data = pd.read_csv('Amazon_Unlocked_Mobile.csv')

In [ ]:
# Clean the NAN data in reviews and brand name
raw_data = raw_data.dropna(subset=['Reviews', 'Brand Name'])
raw_data = raw_data.reset_index(drop=True)
print(len(raw_data))
print(raw_data['Reviews'].isnull().sum())
print(raw_data['Brand Name'].isnull().sum())

In [ ]:
import re
# Get data & labels
reviews = raw_data['Reviews']
# print(reviews)
reviews = reviews.astype(str)
labels = raw_data['Rating']
labels.replace({1:0, 2:0, 3:1, 4:2, 5:2}, inplace=True)
brands = list(zip(raw_data['Brand Name'], raw_data['Reviews']))
df_brands = pd.DataFrame(brands, columns=['Brand Name', 'Reviews'])
print(df_brands)

In [ ]:
print(labels.value_counts())

In [ ]:
sns.countplot(x='Rating', data=raw_data)
plt.title('Rating Distribution')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.show()

# 關鍵字提取


In [ ]:
# Create a DataFrame with only the reviews
df_reviews = pd.DataFrame(reviews, columns=['Reviews'])

# Example output
print(df_reviews)

#rake

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Initialize RAKE with English stopwords
rake = Rake()

In [ ]:
def extract_top_keywords_rake(review):
    rake.extract_keywords_from_text(review)
    # Get the ranked phrases and scores
    ranked_phrases_with_scores = rake.get_ranked_phrases_with_scores()
    # Return the top 5 keywords (or phrases)
    top_keywords = [phrase for score, phrase in ranked_phrases_with_scores[:5]]
    return top_keywords

In [ ]:
# Enable the progress bar
tqdm.pandas()

# Apply RAKE keyword extraction to the reviews
df_reviews['Top Keywords'] = df_reviews['Reviews'].progress_apply(extract_top_keywords_rake)

# Example output
print(df_reviews[['Reviews', 'Top Keywords']].head())
print(df_reviews['Top Keywords'][9])

#TF-IDF

In [ ]:
import nltk

# Download the 'stopwords' dataset
nltk.download('stopwords')

# Download the tokenizer resources for POS tagging
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import re
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize

# Ensure you have downloaded nltk resources (run once)
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

# Sample DataFrame with reviews (replace with your actual data)
df_reviews = pd.DataFrame({'Reviews': [...]})

# Define your custom stop words
nltk_stop_words = set(stopwords.words('english'))
custom_stop_words = list(nltk_stop_words.union({
    'is', 'a', 'the', 'and', 'for', 'to', 'do', 'not', 'your'  # Add more words as needed
}))

# Text preprocessing function to retain adjectives
def preprocess_and_extract_adjectives(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens)  # POS tagging
    adjectives = [word for word, pos in tagged if pos.startswith('JJ')]  # Retain adjectives (JJ = adjective)
    return ' '.join(adjectives)

# Apply the preprocessing function to each review
df_reviews['Cleaned Reviews'] = df_reviews['Reviews'].apply(preprocess_and_extract_adjectives)
df_reviews['Cleaned Reviews'].fillna('', inplace=True)
# Initialize the TF-IDF Vectorizer with custom stop words and n-grams
tfidf_vectorizer = TfidfVectorizer(stop_words=custom_stop_words, ngram_range=(1, 2))

# Fit and transform the cleaned reviews into TF-IDF scores
tfidf_matrix = tfidf_vectorizer.fit_transform(df_reviews['Cleaned Reviews'])

# Get feature names (the important words)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Create a DataFrame for the TF-IDF matrix
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# Extract the top keywords with a parameter for the number of keywords
def extract_top_keywords(row, n=5):
    return row.nlargest(n).index.tolist()

# Use tqdm to show progress
tqdm.pandas(desc="Extracting Top Keywords")
df_reviews['Top Keywords'] = tfidf_df.progress_apply(lambda x: extract_top_keywords(x), axis=1)

# Show the reviews and their top keywords
print(df_reviews[['Reviews', 'Top Keywords']])


#TextRank

In [ ]:
import pandas as pd
import spacy
import pytextrank

In [ ]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank")

In [ ]:
def extract_keywords_textrank(review):
    doc = nlp(review)
    keywords = [phrase.text for phrase in doc._.phrases[:5]]  # Get top 5 keywords
    return keywords

In [ ]:
df_reviews['Top Keywords'] = df_reviews['Reviews'].apply(extract_keywords_textrank)

# Example output
print(df_reviews[['Reviews', 'Top Keywords']].head())

#**Combine** ⬇⬇⬇

In [ ]:
# Combine Reviews and Top Keywords into a single string
df_reviews['Combined'] = df_reviews.apply(lambda x: f"{x['Reviews']} {' '.join(x['Top Keywords'])}", axis=1)
reviews = df_reviews['Combined']

# Vectorize the combined text using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df_reviews['Combined'])
print(df_reviews['Combined'][0])

# 提取關鍵句子

In [ ]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
class CommentAnalyzer:
    def __init__(self, model_name='paraphrase-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name)

        # Initialize the summarization pipeline
        self.summarizer = pipeline("summarization")

    def get_sentence_embeddings(self, sentences):
        # Compute sentence embeddings
        return self.model.encode(sentences, show_progress_bar=True)

    def get_comment_embedding(self, sentence_embeddings):
        # Compute the mean of the sentence embeddings (representing the whole comment)
        comment_embedding = sentence_embeddings.mean(axis=0)
        return comment_embedding

    def most_important_sentence(self, comment_series):
        """
        Extract the most important sentence for each comment in a Pandas Series.

        Parameters:
        - comment_series: A Pandas Series where each element is a comment (string).

        Returns:
        - A Pandas Series containing the most important sentence for each comment.
        """
        most_important_sentences = []

        # Split each comment into sentences
        all_sentences = comment_series.apply(lambda comment: comment.split('. '))

        # Flatten all sentences
        flat_sentences = [sentence for sentences in all_sentences for sentence in sentences]
        # Keep track of sentence numbers for each comment
        sentence_count_per_comment = [len(sentences) for sentences in all_sentences]

        # Get embeddings for all sentences at once (in batch mode)
        sentence_embeddings = self.get_sentence_embeddings(flat_sentences)

        # Initialize the index pointer of a specific sentence
        idx = 0
        most_important_sentences = []

        # Find the most important sentence of each comment according to sentence embeddings
        # Process each comment with progress bar
        for count in tqdm(sentence_count_per_comment, desc="Processing Comments", unit="comment"):
            # Extract the embeddings for the current comment's sentences
            comment_sentence_embeddings = sentence_embeddings[idx: idx + count]

            # Compute the comment embedding (mean of the sentence embeddings)
            comment_embedding = self.get_comment_embedding(comment_sentence_embeddings)

            # Compute cosine similarities
            similarities = cosine_similarity(comment_sentence_embeddings, comment_embedding.reshape(1, -1)).flatten()

            # Find the most important sentence (with the highest similarity)
            most_important_idx = similarities.argmax()
            most_important_sentence = flat_sentences[idx + most_important_idx]
            most_important_sentences.append(most_important_sentence)

            # Move to the next comment's sentences
            idx += count

        # Return the most important sentences as a Pandas Series
        return pd.Series(most_important_sentences, index=comment_series.index)


    def summarize_comment(self, comment):
        # Preprocess the comment
        cleaned_comment = comment.strip()
        # cleaned_comment = comment.apply(lambda x: x.strip())

        # Generate the summary
        summary = self.summarizer(cleaned_comment, max_length=50, min_length=25, do_sample=False)
        return summary[0]['summary_text']


In [ ]:
analyzer = CommentAnalyzer()
s = analyzer.summarize_comments(df_brands['Reviews'])
# print(s)
print(df_brands)
df_brands_new = analyzer.most_important_sentence(df_brands['Reviews'])
print(df_brands_new)

In [ ]:
combined_string = df_brands_new.str.cat(sep=' ')
# print(combined_string)

# Create a new pd.Series with the combined string
combined_series = pd.Series([combined_string])
final_comment = analyzer.most_important_sentence(combined_series)
print(final_comment)

In [ ]:
comment = analyzer.summarize_comment(final_comment)
print(final_comment[0])
print(comment[0])

# 下載資料後以 tokenizer 進行資料處理


In [ ]:
num_train = 244031
num_val = 104585

In [ ]:
def get_data(mode='train'):
    data = []
    Y = []

    if mode == 'train':
        # Combine labels and reviews using zip
        combined = zip(labels[:num_train], reviews[:num_train])
    else:
        # Combine labels and reviews for validation set
        combined = zip(labels[num_train:num_train+num_val], reviews[num_train:num_train+num_val])

    # Filter out entries with NaN labels and separate data and labels
    for y, review in combined:
        if not np.isnan(y):
            data.append(review)
            Y.append(y)

    return data, Y

In [ ]:
# Loading training data & validation data
train_data, train_labels = get_data('train')

In [ ]:
val_data, val_labels = get_data(mode='val')

In [ ]:
print('Number of training:', len(train_data))
print('Number of validation:', len(val_data))

In [ ]:
print(train_data[0])
print(train_labels[0])

In [ ]:
# Initialize a BERT tokenizer from the transformers library
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def preprocessing(texts, labels, mode='train', batch_size=32):

    tokens = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
    tokens = {key: val.to(device) for key, val in tokens.items()}
    labels_tensor = torch.tensor(labels).to(device)
    return {
        'input_ids': tokens['input_ids'],
        'attention_mask': tokens['attention_mask'],
        'labels': labels_tensor
  }

In [ ]:
batch_size = 32

In [ ]:
# Loading data
train_data = preprocessing(train_data, train_labels)
val_data = preprocessing(val_data, val_labels)

In [ ]:
print(train_data['labels'].min(), labels.max())  # 應該輸出 0 和 2

print(train_data['input_ids'].device)
print(train_data['attention_mask'].device)
print(train_data['labels'].device)

print(train_data)
print(type(train_data['input_ids']))  # 應該是 torch.Tensor
print(type(train_data['attention_mask']))  # 應該是 torch.Tensor
print(type(train_data['labels']))  # 應該是 torch.Tensor

In [ ]:
# Create TensorDataset
train_dataset = TensorDataset(train_data['input_ids'], train_data['attention_mask'], train_data['labels'])
val_dataset = TensorDataset(val_data['input_ids'], val_data['attention_mask'], val_data['labels'])

In [ ]:
# Create DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
for batch in train_dataloader:
    input_ids, attention_mask, labels = batch
    print(input_ids.device)
    print("input_ids shape:", input_ids.shape)
    print("attention_mask shape:", attention_mask.shape)
    print("labels shape:", labels.shape)
    break  # 只查看第一個批次的形狀

# 模型訓練

In [ ]:
print_every, num_classes = 3200, 3

In [ ]:
# Define the model
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)

In [ ]:
num_epochs = 2

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=3e-5)

In [ ]:
def plot_training_progress(train_losses, val_losses, train_accuracies, val_accuracies):
    # 設置圖像大小
    plt.figure(figsize=(14, 10))

    # 繪製訓練和驗證損失曲線
    plt.subplot(2, 1, 1)
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Training and Validation Loss per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # 繪製訓練和驗證準確率曲線
    plt.subplot(2, 1, 2)
    plt.plot(train_accuracies, label='Training Accuracy')
    plt.plot(val_accuracies, label='Validation Accuracy')
    plt.title('Training and Validation Accuracy per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # 顯示圖像
    plt.tight_layout()
    plt.show()


In [ ]:
def evaluate_predictor(model, epoch, dataloader, device, loss_fn):
    model.eval()
    total_correct = 0
    total_samples = 0
    total_loss = 0.0  # 用來累計總損失

    with torch.no_grad():
        for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1} Evaluation"):
            input_ids, attention_mask, labels = [b.to(device) for b in batch]

            # 模型預測
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            scores = outputs.logits  # 取模型輸出的 logits
            predictions = scores.argmax(dim=1)  # 獲取預測結果

            # 計算損失
            loss = loss_fn(scores, labels)  # 計算 batch 的損失
            total_loss += loss.item()  # 將損失累加到總損失

            # 計算準確率
            total_correct += (predictions == labels).sum().item()
            total_samples += labels.size(0)

    # 計算整個驗證集的平均損失和準確率
    avg_loss = total_loss / len(dataloader)
    accuracy = total_correct / total_samples

    print(f'Epoch [{epoch + 1}] Accuracy: {accuracy:.4f}, Validation Loss: {avg_loss:.4f}')
    return accuracy, avg_loss


In [ ]:
def train(num_epochs, model, train_dataloader, val_dataloader, device, optimizer):
    scaler = GradScaler()  # 初始化 GradScaler 用於混合精度訓練
    num_iters = 0
    # 確保模型在正確設備上
    model.to(device)

    # loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)
    loss_fn = torch.nn.CrossEntropyLoss()


     # 用於記錄損失和準確率的列表
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    for epoch in range(num_epochs):
        # Training loop
        model.train()
        total_train_loss = 0
        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1} Training"):
            # 將所有張量移動到 GPU
            input_ids = batch[0]
            attention_mask = batch[1]
            labels = batch[2]

            optimizer.zero_grad()

            with autocast():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = loss_fn(outputs.logits, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)  # 更新權重（跳過任何 Nan 梯度）
            scaler.update()

            total_train_loss += loss.item()
            # Validation loop
            if num_iters % print_every == 0:
                evaluate_predictor(model, epoch, val_dataloader, device, loss_fn)
            num_iters += 1


        # 驗證每個 epoch 的 train 準確率跟 loss
        train_accuracy, train_loss = evaluate_predictor(model, epoch, train_dataloader, device, loss_fn)
        train_accuracies.append(train_accuracy)
        train_losses.append(train_loss)

        # 驗證每個 epoch 的 val 準確率跟 loss
        val_accuracy, val_loss = evaluate_predictor(model, epoch, val_dataloader, device, loss_fn)
        val_accuracies.append(val_accuracy)
        val_losses.append(val_loss)
    plot_training_progress(train_losses, val_losses, train_accuracies, val_accuracies)


In [ ]:
# Start training
train(num_epochs, model, train_dataloader, val_dataloader, device, optimizer)

In [ ]:
dataset_name = 'XXX'
saved_model_path = './{}_bert.pth'.format(dataset_name.replace('/', '_'))

# 保存模型權重
torch.save(model.state_dict(), saved_model_path)
print(f"Model weights saved at {saved_model_path}")

In [ ]:
# 加載模型架構和權重
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
# device = torch.device("cpu")
model.to(device)

# 加載訓練好的權重
saved_model_path = 'Amazon_Unlocked_Mobile.csv_bert (1).pth'
model.load_state_dict(torch.load(saved_model_path))

# 切換模型到評估模式
# model.to(device)
model.eval()

print("Model loaded successfully and ready for inference.")

# 開始預測

In [ ]:
# 預處理文本
def preprocess_text(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
    return encoding['input_ids'], encoding['attention_mask']

# 進行預測
def predict_sentiment(model, examples):
    model.eval()
    results = []
    with torch.no_grad():
        for example in examples:
            input_ids, attention_mask = preprocess_text(example)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)

            # 模型預測
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            # print(logits)

            # 使用 softmax 獲取每個類別的概率
            probabilities = torch.softmax(logits, dim=1)
            predicted_class = probabilities.argmax(dim=1)  # 獲取預測的類別

            results.append(predicted_class.item())  # 獲取預測類別索引

    return results

In [ ]:
def print_predicted_result(inputs, results):
    result_for_printing = [
        f'input: {inputs[i]:<30} : score: {results[i]:.1f}'
        for i in range(len(inputs))
    ]
    print(*result_for_printing, sep='\n')
    # print()

In [ ]:
macrumors_data = pd.read_csv('MacRumors - MacRumors.csv')
# 轉換日期格式
date = pd.to_datetime(macrumors_data['time'], format='%b %d, %Y')
mr_after_data = macrumors_data[date > '2024-09-10']
mr_before_data = macrumors_data[(date >= '2024-08-01') & (date <= '2024-09-10')]

mr_after_contents = mr_after_data['content'].astype(str).tolist()
mr_before_contents = mr_before_data['content'].astype(str).tolist()

# 使用加載的模型進行預測
mr_predicted_results_after = predict_sentiment(model, mr_after_contents)
mr_predicted_results_before = predict_sentiment(model, mr_before_contents)

# 打包預測結果
mr_after_data['predicted_result'] = mr_predicted_results_after
mr_before_data['predicted_result'] = mr_predicted_results_before

In [ ]:
trustpilot_data = pd.read_csv('Trustpilot_commemt_data.csv')
# 轉換日期格式
date = pd.to_datetime(trustpilot_data['time'], format='%B %d, %Y')
tp_after_data = trustpilot_data[date > '2024-09-10']
tp_before_data = trustpilot_data[(date >= '2024-08-01') & (date <= '2024-09-10')]

tp_after_contents = tp_after_data['content'].astype(str).tolist()
tp_before_contents = tp_before_data['content'].astype(str).tolist()

# 使用加載的模型進行預測
tp_predicted_results_after = predict_sentiment(model, tp_after_contents)
tp_predicted_results_before = predict_sentiment(model, tp_before_contents)

# 打包預測結果
tp_after_data['predicted_result'] = tp_predicted_results_after
tp_before_data['predicted_result'] = tp_predicted_results_before

In [ ]:
apir_data = pd.read_csv('APIR_commemt_data.csv')
# 轉換日期格式
date = pd.to_datetime(apir_data['time'], format='%B %d, %Y %I:%M%p')
ap_after_data = apir_data[date > '2024-09-10']
ap_before_data = apir_data[(date >= '2024-08-01') & (date <= '2024-09-10')]

ap_after_contents = ap_after_data['content'].astype(str).tolist()
ap_before_contents = ap_before_data['content'].astype(str).tolist()

# 使用加載的模型進行預測
ap_predicted_results_after = predict_sentiment(model, ap_after_contents)
ap_predicted_results_before = predict_sentiment(model, ap_before_contents)

# 打包預測結果
ap_after_data['predicted_result'] = ap_predicted_results_after
ap_before_data['predicted_result'] = ap_predicted_results_before

In [ ]:
mr_after_data.to_csv('mr_after_data (predicted)_BERT_0801-0910.csv', index=False)
mr_before_data.to_csv('mr_before_data (predicted)_BERT_0801-0910.csv', index=False)

tp_after_data.to_csv('tp_after_data (predicted)_BERT_0801-0910.csv', index=False)
tp_before_data.to_csv('tp_before_data (predicted)_BERT_0801-0910.csv', index=False)

ap_after_data.to_csv('ap_after_data (predicted)_BERT_0801-0910.csv', index=False)
ap_before_data.to_csv('ap_before_data (predicted)_BERT_0801-0910.csv', index=False)

In [ ]:
# 讀取六個 CSV 文件
df1 = pd.read_csv('mr_after_data (predicted)_BERT_0801-0910.csv')
df2 = pd.read_csv('mr_before_data (predicted)_BERT_0801-0910.csv')
df3 = pd.read_csv('tp_after_data (predicted)_BERT_0801-0910.csv')
df4 = pd.read_csv('tp_before_data (predicted)_BERT_0801-0910.csv')
df5 = pd.read_csv('ap_after_data (predicted)_BERT_0801-0910.csv')
df6 = pd.read_csv('ap_before_data (predicted)_BERT_0801-0910.csv')


# 提取 Review 和 Time 列，合併 DataFrame
combined_df = pd.concat(
    [df1[['content', 'time', 'predicted_result']],
     df2[['content', 'time', 'predicted_result']],
     df3[['content', 'time', 'predicted_result']],
     df4[['content', 'time', 'predicted_result']],
     df5[['content', 'time', 'predicted_result']],
     df6[['content', 'time', 'predicted_result']]
     ], ignore_index=True)


# 創建新的 DataFrame 保存合併後的结果
result_df = pd.DataFrame({
    'Content': combined_df['content'],
    'Time': combined_df['time'],
    'Predicted_Result': combined_df['predicted_result']
})

# 將结果保存到新的 CSV 文件
result_df.to_csv('merged_3_sources_content_time(predicted)_BERT_0801start.csv', index=False)

print("六個文件已成功合併！")

# 設定評分標準

In [ ]:
# 品牌聲譽評分設定
def calculate_score(predicted_result):
    nominator = 0
    denominator = 0

    denominator = len(predicted_result)
    positive_count = sum(1 for result in predicted_result if result == 2)
    negative_count = sum(1 for result in predicted_result if result == 0)
    nominator = positive_count - negative_count

    return nominator / denominator * 100

In [ ]:
mr_before_score = calculate_score(mr_predicted_results_before)
mr_after_score = calculate_score(mr_predicted_results_after)

tp_before_score = calculate_score(tp_predicted_results_before)
tp_after_score = calculate_score(tp_predicted_results_after)

ap_before_score = calculate_score(ap_predicted_results_before)
ap_after_score = calculate_score(ap_predicted_results_after)

after_df = combined_df[combined_df['time'] > '2024-09-10']
before_df = combined_df[(combined_df['time'] >= '2024-08-01') & (combined_df['time'] <= '2024-09-10')]
print(before_df)

# 然後對篩選後的數據計算分數
total_after_score = calculate_score(after_df['predicted_result'])
total_before_score = calculate_score(before_df['predicted_result'])

print(mr_before_score)
print(mr_after_score)

print(tp_before_score)
print(tp_after_score)

print(ap_before_score)
print(ap_after_score)

print(total_before_score)
print(total_after_score)

# 生成文字雲

In [ ]:
# 讀取停用詞的 .txt 文件，將每行讀取為一個詞彙
with open('stopwords.txt', 'r') as f:
    custom_stopwords = [word.strip() for word in f.read().splitlines()]

total_predicted_results = pd.read_csv('merged_3_sources_content_time(predicted)_BERT_0801start.csv')
total_predicted_results['Time'] = pd.to_datetime(total_predicted_results['Time'], errors='coerce')
before_df = total_predicted_results[(total_predicted_results['Time'] >= '2024-08-01') & (total_predicted_results['Time'] <= '2024-09-10')]
after_df = total_predicted_results[total_predicted_results['Time'] > '2024-09-10']

before_df = before_df.dropna(subset=['Content'])
after_df = after_df.dropna(subset=['Content'])

# 提取關鍵詞、計算詞頻
vectorizer = CountVectorizer(stop_words=custom_stopwords)
after_vector = vectorizer.fit_transform(after_df['Content'])
word_freq = np.asarray(after_vector.sum(axis=0)).flatten()
words = vectorizer.get_feature_names_out()
word_freq_dict = dict(zip(words, word_freq))

# 使用 WordCloud 生成文字雲
wordcloud = WordCloud(
    font_path='Lora-Medium-5.ttf',  # 字體路徑
    colormap='magma',               # 設置顏色映射
    width=800,                      # 設置寬度
    height=400                      # 設置高度
).generate_from_frequencies(word_freq_dict)

# 顯示文字雲
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

before_vector = vectorizer.fit_transform(before_df['Content'])
word_freq = np.asarray(before_vector.sum(axis=0)).flatten()
words = vectorizer.get_feature_names_out()
word_freq_dict = dict(zip(words, word_freq))

# 使用 WordCloud 生成文字雲
wordcloud = WordCloud(
    font_path='Lora-Medium-5.ttf',  # 字體路徑
    colormap='magma',               # 設置顏色映射
    width=800,                      # 設置寬度
    height=400                      # 設置高度
).generate_from_frequencies(word_freq_dict)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

#視覺化
1. 輸入評論，就能看到該評論的情感分析結果
2. 選擇某品牌 & 某個時間段，就能看到該時間段的品牌聲譽評分變化（折線圖）

In [ ]:
# 設置 Gradio 接口
iface = gr.Interface(
    fn=sentiment_score,
    inputs=gr.Textbox(label="輸入評論"),
    outputs=gr.Label(label="情感分析結果")
)

iface2 = gr.Interface(
    fn=calculate_reputation_score,
    inputs=[gr.Number(label="正面評論數量"),
            gr.Number(label="負面評論數量"),
            gr.Number(label="中立評論數量")],
    outputs=gr.Number(label="品牌聲譽評分")
)

# 運行 Gradio 接口
iface.launch()
iface2.launch()

# 分開處理每月的評論

In [ ]:
monthly_data = pd.read_csv('Extracted_Review_Data.csv')

In [ ]:
unique_months = np.sort(monthly_data['Month'].unique())
unique_years = np.sort(monthly_data['Year'].unique())

In [ ]:
def calculate_score(predicted_result):
    nominator = 0
    denominator = len(predicted_result)

    positive_count = sum(1 for result in predicted_result if result == 2)
    negative_count = sum(1 for result in predicted_result if result == 0)
    nominator = positive_count - negative_count

    return nominator / denominator * 100 if denominator > 0 else 0

In [ ]:
def preprocess_text_batch(texts, tokenizer, max_length=128):
    encodings = tokenizer.batch_encode_plus(
        texts,
        add_special_tokens=True,
        max_length=max_length,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
    return encodings['input_ids'], encodings['attention_mask']

In [ ]:
def predict_sentiment(model, examples, tokenizer, batch_size=16):
    model.to(device)
    model.eval()
    results = []

    # 將文本分批處理
    for i in range(0, len(examples), batch_size):
        batch_examples = examples[i:i + batch_size]

        # 預處理文本批次
        input_ids, attention_mask = preprocess_text_batch(batch_examples, tokenizer)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        with torch.no_grad():
            # 模型預測
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # 使用 softmax 獲取每個類別的概率
            probabilities = torch.softmax(logits, dim=1)
            predicted_classes = probabilities.argmax(dim=1)  # 獲取預測的類別

            # 將結果轉換為列表，並加入結果
            results.extend(predicted_classes.cpu().numpy())

    return results

In [ ]:
def get_share_of_mind(unique_years, unique_months, monthly_data, model, tokenizer):
    data = {}
    for unique_year in unique_years:
        data[unique_year] = {}
        data[unique_year]['month'] = []
        data[unique_year]['share_of_mind'] = []
        for unique_month in unique_months:
            if unique_year == 2024 and unique_month > 9:
                break
            else:
                month_data = monthly_data[(monthly_data['year'] == unique_year) & (monthly_data['month'] == unique_month)]
                if month_data.empty:
                    continue
                data[unique_year]['month'].append(unique_month)
                contents = month_data['Merged_Review'].astype(str).tolist()
                predicted_results = predict_sentiment(model, contents, tokenizer)
                score = calculate_score(predicted_results)
                data[unique_year]['share_of_mind'].append(score)
                print(f"{unique_year}-{unique_month}: {score}")
    return data


In [ ]:
data = get_share_of_mind(unique_years, unique_months, monthly_data, model, tokenizer)

In [ ]:
df = pd.DataFrame(data)
df.to_csv('results.csv', index=False)